In [ ]:
from pynq import Overlay
import os
import sys

HOP_DIR=os.path.abspath("../")
sys.path.insert(0, HOP_DIR)
import pushpush

OVERLAY_DIR=f'{HOP_DIR}/overlays/a_add/'

# Preparation

#### Load the overlay onto the FPGA

In [ ]:
ol = Overlay(OVERLAY_DIR + "a_add.bit")

#### Register the overlay with the HoP context

In [ ]:
context = pushpush.Context(ol)

# Use

#### Create some python functions

In [ ]:
def py_func1() -> int:
    return 58

def py_func2() -> int:
    return 1000

def py_func3() -> int:
    return 100

#### Register functions with HoP context

In [ ]:
a_py = context.register(py_func1, "() -> b32")
b_py = context.register(py_func2, "() -> b32")
c_py = context.register(py_func3, "() -> b32")
d_constant = context.register(1000, 'b32')

In [ ]:
context.print_all_objects()

#### Get our hardware

In [ ]:
a_hw = context.functions['hardware']['a']
add = context.functions['hardware']['add']

#### Use Hardware as a regular function
You can mix hardware, functions, variables

In [ ]:
x = add(a_hw, a_py)
print(x)
y = add(add(a_py, b_py), add(a_py, d_constant))
print(y)
z = add(a_hw, 6)
print(x)

# Debug

In [ ]:
import importlib as il
if 'context' in locals():
    del context
    il.reload(pushpush)
    pushpush.Context.reloadModules()
    context = pushpush.Context(ol)